<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Ronen_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, place the data files somewhere in your drive so that you can access the files from this notebook. The files are available to download from the Moodle assignment activity*

The relevant files are:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)





**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from pathlib import Path
from glob import glob
import csv
import json
import pandas as pd

def preprocess():
  # TODO
 vocabulary = set()
 pathlist = glob('/content/drive/My Drive/IDC/NLP/Assignment1/*.csv')
 for path in pathlist:
    with open(path, encoding='utf-8', newline='') as csv_file:
        csv_reader = pd.read_csv(csv_file, delimiter=',')
        line_count = 0
        for text in csv_reader['tweet_text']:
            list_text = list(text)
            vocabulary.update(list_text)
            line_count += 1
        #print(line_count)
 return sorted(list(vocabulary))

vocabulary = preprocess()
print(len(vocabulary))
print(vocabulary)
  

1859
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x91', '\x92', '\x97', '\x9d', '¡', '£', '¤', '¥', '§', '¨', '©', 'ª', '«', '\xad', '®', '¯', '°', '²', '³', '´', '¶', '·', '¸', 'º', '»', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Å', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ù', 'Ú', 'Ü', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ė', 'Ğ', 'ğ', 'İ', 'ı', 'ń', 'ō', 'Œ', 'œ', 'Ş', 'ş', 'Š', 'Ÿ', 'ƒ', 'ʔ', 'ʕ', 'ʖ', 'ʰ', 'ʳ', '

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [3]:
import nltk.data
import nltk.tokenize 
from collections import Counter
def find_ngrams(text, length):
    counter = Counter()
    ngram = {}
    sequence  = {}
    num_tokens, tokens = split_into_tokens(text.lower())
    for token in tokens:
        for i,_ in enumerate(token):
           if(i>=length - 1):
                post = token[i]
                prior = token[i - (length - 1) : i]
                word = ''.join(prior) + post
                counter[word] = counter.get(word, 0) + 1
                sequence[post] = counter[word]
                ngram[prior] = sequence
    return ngram
def token_to_unigram(token):
    token = token.strip().strip(",.!|&-_()[]<>{}/\"'").strip()

    def has_no_chars(token):
        for char in token:
            if char.isalpha():
                return False
        return True

    if len(token) == 1 or token.isdigit() or has_no_chars(token):
        return None
    return token
def split_into_tokens(text):
    tokens = []
    for token in nltk.tokenize.WhitespaceTokenizer().tokenize(text):
    #for token in tokenize(text):
        unigram = token_to_unigram(token)
        if unigram:
            tokens.append(unigram)
    return len(tokens), tokens
def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  # TODO
  model = {}
  with open(data_file_path, encoding='utf-8', newline='') as csv_file:
    csv_reader = pd.read_csv(csv_file, delimiter=',')
    corpus = []
    for tweet_text in csv_reader['tweet_text']:
            corpus.append(tweet_text)
    data = find_ngrams(" ".join(corpus),n)

    for word,chars in data.items():
        temp_dict = {}
        words_total = float(sum(chars.values()))
        for char, count in chars.items():
            if (add_one == True):
                p = float(count + 1)/(words_total + len(vocabulary))
            else:
                p = float(count)/words_total
            temp_dict[char] = p
        model[word] = temp_dict
  return model
n =3
file_name = '/content/drive/My Drive/IDC/NLP/Assignment1/en.csv'
model = lm(n, vocabulary,file_name , True)
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
for j, key in enumerate(model.keys()):
    if j < 50:
        print("ngram:{:s} {:f}".format(key,sum(model[key].values())))  

{'@h': {'a': 0.02884311951696716, 'n': 0.00010519007847179855, 'h': 6.311404708307913e-05, 'm': 0.01009824753329266, 'e': 0.004123451076094502, 'o': 0.12084236214840216, 'r': 8.415206277743884e-05, 'i': 0.008583510403298761, 's': 0.12502892727157974, '1': 4.207603138871942e-05, ':': 0.12467128100477563, 't': 0.12275682157658889, 'l': 0.018345149685481666, 'y': 0.005007047735257611, 'c': 4.207603138871942e-05, 'p': 4.207603138871942e-05, 'f': 0.0008415206277743884, 'u': 0.0592640902110113, '/': 0.12019018366187702, '.': 0.12176803483895399, 'd': 0.049397260850356596, '9': 0.0014726610986051795, 'z': 0.0012412429259672229, '_': 0.0005890644394420719, 'g': 0.00012622809416615825, 'b': 4.207603138871942e-05, 'w': 0.0003155702354153956, 'k': 0.012370353228283509, "'": 0.006521784865251509, '…': 0.00016830412555487767, '~': 4.207603138871942e-05, '?': 0.00014726610986051796, '3': 0.000357646266804115, '7': 8.415206277743884e-05, '8': 6.311404708307913e-05, 'v': 0.0004207603138871942, 'x': 8.

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [4]:
import numpy as np
def compute_entropy_xi(ngram):
    ll = 0
    for key in ngram.items():
        p = key[1]
        ll += p*np.log2(p)
    return ll

def eval(n, model, data_file):
    """
    Compute the perplexity of ngrams from model
    """
    h_x = 0
    N = 0
    with open(data_file, encoding='utf-8', newline='') as csv_file:
        csv_reader = pd.read_csv(csv_file, delimiter=',')
        corpus = []
        for tweet_text in csv_reader['tweet_text']:
                corpus.append(tweet_text)
        data = find_ngrams(" ".join(corpus),n)
        N = len(model)   # Total number of words
        for word,chars in data.items():
            if word in model:
                ngram = model[word]
                #for i, ngram in enumerate(model.items()):
                h_x += compute_entropy_xi(ngram)
            #N += len(h_X)
    return pow(2, (-1.0 / N) * h_x)

test_file_name = '/content/drive/My Drive/IDC/NLP/Assignment1/es.csv'
preplexity = eval(n,model,test_file_name)
print("Preplexity evaluation : {:f}".format(preplexity))

Preplexity evaluation : 5.128441


**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [0]:
import os
def match(n, add_one):
  # n - the n-gram to use for creating n-gram models
  # add_one - use add_one smoothing or not

  #TODO
 path_name = '/content/drive/My Drive/IDC/NLP/Assignment1/'
 csvglob = path_name + '*.csv'
 pathlist = glob(csvglob) #'C:\\Users\\ronen\\Documents\\IDC\Year-3\\Spring 2020\\NLP\\HW\\Assignment1\\*.csv')
 num = len(pathlist)
 matrix = np.empty(shape=(num,num),dtype='float')
 headers_list = []
 i = 0
 for model_path in pathlist:
    head, tail = os.path.split(model_path)
    headers_list.append(tail)
    j = 0
    modeli = lm(n, vocabulary,model_path, add_one)
    for data_file_path in pathlist:
        #if ( i != j ):
        pp = eval(n,modeli,data_file_path)
        matrix[i][j] = pp
        print ("Preplexity of model {:d} with data file {:d} is {:f}".format(i,j,pp))
        j = j+1
    i = i + 1
 print(matrix)
 dataframe = pd.DataFrame(matrix,columns=headers_list)
 print("passed dataframe")
 return dataframe

**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [0]:


max_n = 4 
for n in range(4): 
  print("n = {:d} add_one:{:b}".format(n+1,False))
  data1 = match(n+1,False) 
  print(data1) 
  print("n = {:d} add_one:{:b}".format(n+1,True))
  data2 = match(n+1,True) 
  print(data2)

n = 1 add_one:0
Preplexity of model 0 with data file 0 is 26.496436
Preplexity of model 0 with data file 1 is 26.496436
Preplexity of model 0 with data file 2 is 26.496436
Preplexity of model 0 with data file 3 is 26.496436
Preplexity of model 0 with data file 4 is 26.496436
Preplexity of model 0 with data file 5 is 26.496436
Preplexity of model 0 with data file 6 is 26.496436
Preplexity of model 0 with data file 7 is 26.496436
Preplexity of model 1 with data file 0 is 25.576839
Preplexity of model 1 with data file 1 is 25.576839
Preplexity of model 1 with data file 2 is 25.576839
Preplexity of model 1 with data file 3 is 25.576839
Preplexity of model 1 with data file 4 is 25.576839
Preplexity of model 1 with data file 5 is 25.576839
Preplexity of model 1 with data file 6 is 25.576839
Preplexity of model 1 with data file 7 is 25.576839
Preplexity of model 2 with data file 0 is 25.984383
Preplexity of model 2 with data file 1 is 25.984383
Preplexity of model 2 with data file 2 is 25.984

# **Good luck!**
max_n = 4
for n = 1 in range(max_n):
  data = match(n,False)
  print(data)
  data = match(n,True)
  print(data)
  